In [ ]:
!pip install tensorflow

In [2]:
import tensorflow as tf

2023-07-15 19:14:09.454380: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-15 19:14:09.553609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-15 19:14:09.555621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 19:14:11.324298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [56]:
birdnet_original_path = "/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model_FP32.tflite"
interpreter = tf.lite.Interpreter(model_path=birdnet_original_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input:", input_details)
print("Output:", output_details)

Input: [{'name': 'INPUT', 'index': 0, 'shape': array([     1, 144000], dtype=int32), 'shape_signature': array([    -1, 144000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output: [{'name': 'Identity', 'index': 491, 'shape': array([   1, 3337], dtype=int32), 'shape_signature': array([  -1, 3337], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [57]:
interpreter = tf.lite.Interpreter(model_path=birdnet_original_path)

layer_details = interpreter.get_tensor_details()
interpreter.allocate_tensors()

for layer in layer_details:
    print("\nLayer Name: {}".format(layer['name']))
    print("\tIndex: {}".format(layer['index']))
    print("\n\tShape: {}".format(layer['shape']))

    try:
        tensor = interpreter.get_tensor(layer['index'])
        if tensor.shape == [0]:
            print("\tTensor: Ignored")
        else:
            print("\tTensor: {}".format(tensor.shape))
    except ValueError as e:
        print("\tError: {}".format(e))
        continue

    print("\tTensor Type: {}".format(tensor.dtype))
    print("\tQuantisation Parameters")
    print("\t\tScales: {}".format(layer['quantization_parameters']['scales'].shape))
    print("\t\tScales Type: {}".format(layer['quantization_parameters']['scales'].dtype))
    print("\t\tZero Points: {}".format(layer['quantization_parameters']['zero_points']))
    print("\t\tQuantized Dimension: {}".format(layer['quantization_parameters']['quantized_dimension']))



Layer Name: INPUT
	Index: 0

	Shape: [     1 144000]
	Tensor: (1, 144000)
	Tensor Type: float32
	Quantisation Parameters
		Scales: (0,)
		Scales Type: float32
		Zero Points: []
		Quantized Dimension: 0

Layer Name: model/POST_BN_1/FusedBatchNormV3;model/POST_CONV_1/Conv2D
	Index: 1

	Shape: [1024]
	Tensor: (1024,)
	Tensor Type: float32
	Quantisation Parameters
		Scales: (0,)
		Scales Type: float32
		Zero Points: []
		Quantized Dimension: 0

Layer Name: model/BLOCK_4-4_BN_2/FusedBatchNormV3;model/BLOCK_4-1_CONV_1/Conv2D;model/BLOCK_4-4_CONV_2/depthwise
	Index: 2

	Shape: [1536]
	Tensor: (1536,)
	Tensor Type: float32
	Quantisation Parameters
		Scales: (0,)
		Scales Type: float32
		Zero Points: []
		Quantized Dimension: 0

Layer Name: model/BLOCK_4-4_BN_2/FusedBatchNormV3;model/BLOCK_4-4_CONV_2/depthwise;model/BLOCK_4-1_CONV_1/Conv2D
	Index: 3

	Shape: [   1    3    3 1536]
	Tensor: (1, 3, 3, 1536)
	Tensor Type: float32
	Quantisation Parameters
		Scales: (0,)
		Scales Type: float32
		Zer

In [5]:
tf.lite.experimental.Analyzer.analyze(model_path=birdnet_original_path,
                                      model_content=None,
                                      gpu_compatibility=False)

=== /home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model_FP32.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the REDUCE_MIN op takes
tensor #0 and tensor #129 as input and produces tensor #158 as output.

Subgraph#0 main(T#0) -> [T#491]
  Op#0 REDUCE_MIN(T#0, T#129[1]) -> [T#158]
  Op#1 SUB(T#0, T#158) -> [T#159]
  Op#2 REDUCE_MAX(T#159, T#129[1]) -> [T#160]
  Op#3 ADD(T#160, T#144) -> [T#161]
  Op#4 DIV(T#159, T#161) -> [T#162]
  Op#5 SHAPE(T#162) -> [T#163]
  Op#6 SPLIT_V(T#163, T#128[1, 1, 0], T#142[0]) -> [T#164, T#165, T#166]
  Op#7 RESHAPE(T#165, T#145[]) -> [T#167]
  Op#8 FLOOR_DIV(T#167, T#146[8]) -> [T#168]
  Op#9 PACK(T#168, T#146[8]) -> [T#169]
  Op#10 MUL(T#168, T#146[8]) -> [T#170]
  Op#11 RESHAPE(T#170, T#135[1]) -> [T#171]
  Op#12 CONCATENATION(T#164, T#171) -> [T#172]
  Op#13 CONCATENATION(T#164, T#169) -> [T#173]
  Op#14 STRIDED_SLIC

In [6]:
index_class = output_details[0]["index"]
print("Outputlayer (Classification) Index:", index_class)

index_embeddings = output_details[0]["index"] -1
print("Outputlayer (Embeddings) Index:", index_embeddings)

Outputlayer (Classification) Index: 491
Outputlayer (Embeddings) Index: 490


In [7]:
interpreter.allocate_tensors()
interpreter.invoke()

output_embeddings = interpreter.get_tensor(index_embeddings)
print("Outputlayer (Embeddings) Output:", output_embeddings)
print("Outputlayer (Embeddings) Size:", len(output_embeddings[0]))

Outputlayer (Embeddings) Output: [[0.1680546  0.07166231 0.05571768 ... 0.33594096 0.19542216 0.39730033]]
Outputlayer (Embeddings) Size: 1024


In [8]:
output_Classification = interpreter.get_tensor(index_class)
print("Outputlayer (Classification) Output:", output_Classification)
print("Outputlayer (Classification) Size:", len(output_Classification[0]))

Outputlayer (Classification) Output: [[ -8.852373   -9.418921   -7.5134993 ...  -9.893699   -8.044197
  -10.668461 ]]
Outputlayer (Classification) Size: 3337


### Build own classifier with two classes
---

In [9]:
from tensorflow import keras

# Build a simple two-layer linear classifier
custom_classifier = keras.Sequential()

# Input layer with 1024 input size
custom_classifier.add(keras.layers.InputLayer(input_shape=1024))

# Hidden layer
custom_classifier.add(keras.layers.Dense(3, activation='relu'))

# Classification layer with number of classes
custom_classifier.add(keras.layers.Dense(2))

# Activation layer
custom_classifier.add(keras.layers.Activation('sigmoid'))

 **Check Shape of self build classifier:**

In [10]:
custom_classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 3075      
                                                                 
 dense_1 (Dense)             (None, 2)                 8         
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 3083 (12.04 KB)
Trainable params: 3083 (12.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**protobuf model convert to keras model**

In [ ]:
proto_model = tf.saved_model.load('/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model')



In [ ]:
proto_model.summary()

**pop layer with tensorflow**

In [47]:
tensor_details = interpreter.get_tensor_details()
last_layer_index = None
for index, tensor in enumerate(tensor_details):
    if tensor['index'] == 491:
        last_layer_index = index
        print("Last Layer with Index " + str(last_layer_index) + " found!")
        break

print("Before removal - tensor_details length:", len(tensor_details))
tensor_details.pop(last_layer_index)
print("After removal - tensor_details length:", len(tensor_details))

interpreter.allocate_tensors()

# Check if tensor with Index 491 is still accessible
try:
    tensor_491 = interpreter.tensor(491)
    print("Tensor with Index 491 still accessible.")
except IndexError:
    print("Tensor with Index 491 is no longer accessible.")

Last Layer with Index 491 found!
Before removal - tensor_details length: 625
After removal - tensor_details length: 624
Tensor with Index 491 still accessible.


NameError: name 'tflite_model' is not defined